In [1]:
import mysql.connector
import pandas as pd

/tmp/ipykernel_15636/4215133018.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
dldb = mysql.connector.Connect(
    host = "server",
    port = "3306",
    user = "username",
    password = "****",
    database = "database",
    use_unicode=True
)

In [11]:
cursor = dldb.cursor()
create_table_query = """
CREATE TABLE IF NOT EXISTS auction_price_data (
        species VARCHAR(16),
        origin VARCHAR(16),
        size VARCHAR(16),
        packaging VARCHAR(16),
        quantity FLOAT,
        highest FLOAT,
        lowest FLOAT,
        average FLOAT,
        date DATE
    )
    """
cursor.execute(create_table_query)


In [5]:

import os
from datetime import datetime, timedelta

start_date = datetime.strptime('2024-02-04', '%Y-%m-%d')
end_date = datetime.strptime('2024-06-06', '%Y-%m-%d')
delta = timedelta(days=1)
cursor = dldb.cursor()
csv_file_template = './{:04d}/{:02d}/{:04d}-{:02d}-{:02d}.csv'
current_date = start_date

while current_date <= end_date:
    csv_file_path = csv_file_template.format(current_date.year, current_date.month, current_date.year, current_date.month, current_date.day)
    if os.path.exists(csv_file_path):
        df = pd.read_csv(csv_file_path, header=None, thousands=',',encoding='utf-8')
        df.columns = ['species', 'origin', 'size', 'packaging', 'quantity', 'highest', 'lowest', 'average', 'date']
        for index, row in df.iterrows():
            insert_query = """
            INSERT INTO auction_price_data (species, origin, size, packaging, quantity, highest, lowest, average, date)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            cursor.execute(insert_query, tuple(row))
        dldb.commit()
    current_date += delta
cursor.close()
dldb.close()